In [ ]:
# 단계 1: 폰트 설치
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm

!apt-get -qq -y install fonts-nanum > /dev/null


# 단계 2: 런타임 재시작
import os
os.kill(os.getpid(), 9)

# 마이너스 표시 문제
mpl.rcParams['axes.unicode_minus'] = False

# 한글 폰트 설정
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 18, 'font.family': 'NanumGothic'}) # 폰트 설정

In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
import matplotlib.pyplot as plt


# 데이터를 DataFrame으로 읽기
df = pd.read_excel('/content/시군별_논벼_생산량.xlsx')

# 첫 두 행을 확인하여 올바른 데이터 구조 파악
print("Initial columns:", df.columns.tolist())
print("First row:", df.iloc[0].tolist())

# 실제 데이터 구조에 맞게 처리
# 두 번째 행을 새로운 열 이름으로 사용
new_columns = []
for col in df.iloc[1]:
    if pd.isna(col):
        new_columns.append('temp')  # 임시 이름 부여
    else:
        new_columns.append(str(col))
df.columns = new_columns

# 첫 두 행 제거 (이미 열 이름으로 사용했으므로)
df = df.iloc[2:].reset_index(drop=True)

# 빈 리스트 생성
processed_data = []

# 각 행정구역별로 데이터 처리
for idx, row in df.iterrows():
    region = row.iloc[0]  # 첫 번째 열이 행정구역

    # 열 이름을 순회하면서 데이터 추출
    for i in range(0, len(row)-1, 2):  # 2칸씩 건너뛰며 처리
        if i+1 < len(row):  # 인덱스 범위 확인
            year = 2008 + (i//2)  # 연도 계산
            area = row.iloc[i]
            production = row.iloc[i+1]

            # 데이터 추가 (문자열 '-' 처리 포함)
            try:
                area_value = float(area) if pd.notnull(area) and str(area) != '-' else 0
                production_value = float(production) if pd.notnull(production) and str(production) != '-' else 0

                processed_data.append({
                    '행정구역': region,
                    '연도': year,
                    '재배면적(ha)': area_value,
                    '생산량(톤)': production_value
                })
            except ValueError:
                continue  # 숫자로 변환할 수 없는 경우 건너뛰기

# 새로운 DataFrame 생성
result_df = pd.DataFrame(processed_data)

# 데이터 정렬
result_df = result_df.sort_values(['행정구역', '연도']).reset_index(drop=True)

# 엑셀로 저장
result_df.to_excel('processed_rice_production.xlsx', index=False)

df1 = pd.read_excel('/content/processed_rice_production.xlsx')
print(df1.head(20))

# 데이터를 DataFrame으로 읽기
df2 = pd.read_csv('/content/연간기후전체지역.csv', encoding='euc-kr')
print(df2.head(20))
# 데이터를 DataFrame으로 읽기
df3 = pd.read_csv('/content/월별기후전체지역.csv', encoding='euc-kr')
print(df3.head(20))









Initial columns: ['행정구역별(1)', '2008', '2008.1', '2009', '2009.1', '2010', '2010.1', '2011', '2011.1', '2012', '2012.1', '2013', '2013.1', '2014', '2014.1', '2015', '2015.1', '2016', '2016.1', '2017', '2017.1', '2018', '2018.1', '2019', '2019.1', '2020', '2020.1', '2021', '2021.1', '2022', '2022.1', '2023', '2023.1']
First row: ['행정구역별(1)', '논벼:재배면적 (ha)', '논벼:생산량 (톤)', '논벼:재배면적 (ha)', '논벼:생산량 (톤)', '논벼:재배면적 (ha)', '논벼:생산량 (톤)', '논벼:재배면적 (ha)', '논벼:생산량 (톤)', '논벼:재배면적 (ha)', '논벼:생산량 (톤)', '논벼:재배면적 (ha)', '논벼:생산량 (톤)', '논벼:재배면적 (ha)', '논벼:생산량 (톤)', '논벼:재배면적 (ha)', '논벼:생산량 (톤)', '논벼:재배면적 (ha)', '논벼:생산량 (톤)', '논벼:재배면적 (ha)', '논벼:생산량 (톤)', '논벼:재배면적 (ha)', '논벼:생산량 (톤)', '논벼:재배면적 (ha)', '논벼:생산량 (톤)', '논벼:재배면적 (ha)', '논벼:생산량 (톤)', '논벼:재배면적 (ha)', '논벼:생산량 (톤)', '논벼:재배면적 (ha)', '논벼:생산량 (톤)', '논벼:재배면적 (ha)', '논벼:생산량 (톤)']
   행정구역    연도  재배면적(ha)  생산량(톤)
0   강원도  2009    198678   40420
1   강원도  2010    209299   38809
2   강원도  2011    176069   35914
3   강원도  2012    170916   34772
4   강원도  2013    1

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [17]:
##processed_rice_production 데이터에서 지역 필터링##

import pandas as pd

# 데이터 읽기
df = pd.read_excel('/content/processed_rice_production.xlsx')

# 원하는 지역 필터링
target_regions = ['전라남도', '충청남도', '전라북도', '경상북도']
filtered_df = df[df['행정구역'].isin(target_regions)]

# 결과 저장
filtered_df.to_excel('filtered_rice_production.xlsx', index=False)

# 결과 확인
print(filtered_df)

     행정구역    연도  재배면적(ha)  생산량(톤)
45   경상북도  2009    658429  122441
46   경상북도  2010    679872  116472
47   경상북도  2011    589986  110458
48   경상북도  2012    569948  111459
49   경상북도  2013    563846  108479
50   경상북도  2014    572166  106815
51   경상북도  2015    570046  104686
52   경상북도  2016    589419  102938
53   경상북도  2017    576636   99550
54   경상북도  2018    549472   98065
55   경상북도  2019    534967   97465
56   경상북도  2020    529210   97255
57   경상북도  2021    495055   95830
58   경상북도  2022    518007   94746
59   경상북도  2023    511363   93252
180  전라남도  2009    888531  183359
181  전라남도  2010    902089  180163
182  전라남도  2011    836939  172842
183  전라남도  2012    823955  171872
184  전라남도  2013    698093  169918
185  전라남도  2014    821354  168763
186  전라남도  2015    806889  169014
187  전라남도  2016    862946  165750
188  전라남도  2017    844611  161192
189  전라남도  2018    826617  154803
190  전라남도  2019    765608  153919
191  전라남도  2020    724643  156026
192  전라남도  2021    687299  155101
193  전라남도  202

In [18]:
##연간기후전체 지역 필터링 코드##

# 데이터 불러오기
import pandas as pd

# 각 도시별 행정구역 정보를 담은 딕셔너리 생성
region_dict = {
    # 전라남도
    '목포': '전라남도', '여수': '전라남도', '순천': '전라남도',
    '완도': '전라남도', '진도(첨찰산)': '전라남도', '진도군': '전라남도',
    '해남': '전라남도', '고흥': '전라남도', '광양시': '전라남도',
    '보성군': '전라남도', '강진군': '전라남도', '장흥': '전라남도',

    # 충청남도
    '서산': '충청남도', '천안': '충청남도', '보령': '충청남도',
    '부여': '충청남도', '홍성': '충청남도',

    # 전라북도
    '전주': '전라북도', '군산': '전라북도', '부안': '전라북도',
    '임실': '전라북도', '정읍': '전라북도', '남원': '전라북도',
    '장수': '전라북도', '고창군': '전라북도', '순창군': '전라북도',

    # 경상북도
    '포항': '경상북도', '안동': '경상북도', '상주': '경상북도',
    '울진': '경상북도', '봉화': '경상북도', '영주': '경상북도',
    '문경': '경상북도', '청송군': '경상북도', '영덕': '경상북도',
    '의성': '경상북도', '구미': '경상북도', '영천': '경상북도',
    '경주시': '경상북도'
}

# 지역 정보 추가
df2['행정구역'] = df2['지점명'].map(region_dict)

# 원하는 지역만 필터링
target_regions = ['전라남도', '충청남도', '전라북도', '경상북도']
filtered_df = df2[df2['행정구역'].isin(target_regions)]

# 결과 출력
print(filtered_df.head())

# 필터링된 데이터 확인
print("\n각 도별 도시 수:")
print(filtered_df['행정구역'].value_counts())


      지점 지점명    일시  평균기온(°C)  합계 강수량(mm)  평균 상대습도(%)  합계 일조시간(hr)  행정구역
280  129  서산  2008      12.0       909.6        74.3       2095.5  충청남도
281  129  서산  2009      12.3      1074.3        73.4       2174.3  충청남도
282  129  서산  2010      11.7      2141.8        73.9       1984.3  충청남도
283  129  서산  2011      11.6      1704.4        73.1       2007.0  충청남도
284  129  서산  2012      11.5      1642.6        75.7       2299.2  충청남도

각 도별 도시 수:
행정구역
경상북도    204
전라남도    173
전라북도    144
충청남도     73
Name: count, dtype: int64
파일이 성공적으로 저장되었습니다.


In [19]:
##월별기후전체 지역 필터링 코드##

# 데이터 불러오기
import pandas as pd

# 각 도시별 행정구역 정보를 담은 딕셔너리 생성
region_dict = {
    # 전라남도
    '목포': '전라남도', '여수': '전라남도', '순천': '전라남도',
    '완도': '전라남도', '진도(첨찰산)': '전라남도', '진도군': '전라남도',
    '해남': '전라남도', '고흥': '전라남도', '광양시': '전라남도',
    '보성군': '전라남도', '강진군': '전라남도', '장흥': '전라남도',

    # 충청남도
    '서산': '충청남도', '천안': '충청남도', '보령': '충청남도',
    '부여': '충청남도', '홍성': '충청남도',

    # 전라북도
    '전주': '전라북도', '군산': '전라북도', '부안': '전라북도',
    '임실': '전라북도', '정읍': '전라북도', '남원': '전라북도',
    '장수': '전라북도', '고창군': '전라북도', '순창군': '전라북도',

    # 경상북도
    '포항': '경상북도', '안동': '경상북도', '상주': '경상북도',
    '울진': '경상북도', '봉화': '경상북도', '영주': '경상북도',
    '문경': '경상북도', '청송군': '경상북도', '영덕': '경상북도',
    '의성': '경상북도', '구미': '경상북도', '영천': '경상북도',
    '경주시': '경상북도'
}

# 지역 정보 추가
df2['행정구역'] = df2['지점명'].map(region_dict)
df3['행정구역'] = df3['지점명'].map(region_dict)

# 원하는 지역만 필터링
target_regions = ['전라남도', '충청남도', '전라북도', '경상북도']
filtered_df = df2[df2['행정구역'].isin(target_regions)]
filtered_df_monthly = df3[df3['행정구역'].isin(target_regions)]

# 결과 출력
print(filtered_df.head())

# 필터링된 데이터 확인
print("\n각 도별 도시 수:")
print(filtered_df['행정구역'].value_counts())

# 필터링된 데이터를 CSV 파일로 저장
filtered_df.to_csv('filtered_climate_data.csv', index=False, encoding='utf-8-sig')
filtered_df_monthly.to_csv('filtered_climate_monthly_data.csv', index=False, encoding='utf-8-sig')

print("파일이 성공적으로 저장되었습니다.")


      지점 지점명    일시  평균기온(°C)  합계 강수량(mm)  평균 상대습도(%)  합계 일조시간(hr)  행정구역
280  129  서산  2008      12.0       909.6        74.3       2095.5  충청남도
281  129  서산  2009      12.3      1074.3        73.4       2174.3  충청남도
282  129  서산  2010      11.7      2141.8        73.9       1984.3  충청남도
283  129  서산  2011      11.6      1704.4        73.1       2007.0  충청남도
284  129  서산  2012      11.5      1642.6        75.7       2299.2  충청남도

각 도별 도시 수:
행정구역
경상북도    204
전라남도    173
전라북도    144
충청남도     73
Name: count, dtype: int64
파일이 성공적으로 저장되었습니다.
